In [2]:
!nvidia-smi

Thu Feb  6 09:54:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install ultralytics

In [4]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.2/112.6 GB disk)


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6095.9/8062.4 GB disk)


In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="uhtPWClvIGgUa3WaB0k6")
project = rf.workspace("ml-lmvau").project("birddetector-wrvb1-v20r0")
version = project.version(1)
dataset = version.download("yolov11")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to BirdDetector-1 in yolov11:: 100%|██████████| 8186/8186 [00:01<00:00, 6310.50it/s]


In [ ]:
dataset.location

'/kaggle/working/BirdDetector-1'

In [ ]:
!yolo task=detect mode=train data={dataset.location}/data.yaml model="yolo11l.pt" epochs=100 imgsz=640 batch=16 plots=True save=True

100%|███████████████████████████████████████| 49.0M/49.0M [00:00<00:00, 226MB/s]
Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=/kaggle/working/BirdDetector-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import os
import cv2
import pandas as pd
from PIL import Image
from tqdm import tqdm
from ultralytics import YOLO

# ===========================
# CONFIGURATION
# ===========================
MODEL_PATH = "/content/drive/MyDrive/bdma-07-competition/best.pt"  # Path to trained YOLO11L model
ROOT_DIR = "/content/drive/MyDrive/bdma-07-competition/BDMA7_project_files/"
SAVE_DIR = "/content/drive/MyDrive/bdma-07-competition/BDMA7_project_files/cropped_images/"
RESIZE_SHAPE = (448, 448)  # Resize cropped images to 448x448

# Paths to metadata files
train_csv = os.path.join(ROOT_DIR, "train_metadata.csv")
val_csv = os.path.join(ROOT_DIR, "val_metadata.csv")
train_img_dir = os.path.join(ROOT_DIR, "train_images")
val_img_dir = os.path.join(ROOT_DIR, "val_images")
test_img_dir = os.path.join(ROOT_DIR, "test_images/mistery_cat")

# ===========================
# LOAD YOLO MODEL
# ===========================
model = YOLO(MODEL_PATH)

# ===========================
# HELPER FUNCTION: CROP, RESIZE, AND SAVE IMAGES
# ===========================
def crop_resize_save_images(image_paths, save_subdir, is_test=False):
    for img_path in tqdm(image_paths, desc=f"Processing {save_subdir} images"):
        try:
            filename = img_path
            if not is_test:
                class_name = filename.split('/')[7]  # Extract class from folder structure
                class_save_dir = os.path.join(SAVE_DIR, save_subdir, class_name)
                os.makedirs(class_save_dir, exist_ok=True)
                save_path = os.path.join(class_save_dir, filename.split('/')[8])  # Keep original name
            else:
                test_save_dir = os.path.join(SAVE_DIR, save_subdir)
                os.makedirs(test_save_dir, exist_ok=True)
                save_path = os.path.join(test_save_dir, filename.split('/')[8])  # Keep original name

            # Read image
            image = cv2.imread(img_path)
            if image is None:
                print(f"Warning: Could not read {img_path}")
                continue

            # Run YOLO inference
            results = model(img_path)

            for result in results:
                boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes [x1, y1, x2, y2]
                if len(boxes) == 0:
                    print(f"Warning: No objects detected in {img_path}, skipping cropping.")
                    continue  # Skip if no objects detected

                # Use the first detected box (assuming one object per image)
                x1, y1, x2, y2 = map(int, boxes[0])

                # Crop the detected region
                cropped_img = image[y1:y2, x1:x2]

                # Convert to PIL format, resize, and save
                cropped_img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
                resized_img_pil = cropped_img_pil.resize(RESIZE_SHAPE, Image.Resampling.LANCZOS)
                resized_img_pil.save(save_path)

        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# ===========================
# PROCESS TRAIN, VAL, TEST IMAGES
# ===========================

# Load train and val metadata
train_metadata = pd.read_csv(train_csv)
val_metadata = pd.read_csv(val_csv)

# Get full image paths
train_images = [os.path.join(train_img_dir, img) for img in train_metadata.iloc[:, 0].tolist()]
val_images = [os.path.join(val_img_dir, img) for img in val_metadata.iloc[:, 0].tolist()]
test_images = [os.path.join(test_img_dir, img) for img in os.listdir(test_img_dir) if img.endswith((".jpg", ".png"))]

print(train_images[0].split("/")[7])

# Process images
crop_resize_save_images(train_images, "train_images")
crop_resize_save_images(val_images, "val_images")
crop_resize_save_images(test_images, "test_images/mistery_cat", is_test=True)

print("✅ Cropped and resized images saved successfully!")

Brandt_Cormorant


Processing train_images images:   0%|          | 0/1082 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/bdma-07-competition/BDMA7_project_files/train_images/Brandt_Cormorant/Brandt_Cormorant_0071_23007.jpg: 640x640 1 Bird, 47.4ms
Speed: 3.0ms preprocess, 47.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


Processing train_images images:   0%|          | 1/1082 [00:00<09:51,  1.83it/s]


image 1/1 /content/drive/MyDrive/bdma-07-competition/BDMA7_project_files/train_images/Brandt_Cormorant/Brandt_Cormorant_0028_22892.jpg: 640x640 1 Bird, 38.5ms
Speed: 3.7ms preprocess, 38.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/bdma-07-competition/BDMA7_project_files/train_images/Brandt_Cormorant/Brandt_Cormorant_0076_23021.jpg: 640x640 1 Bird, 38.4ms
Speed: 2.6ms preprocess, 38.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


...


image 1/1 /content/drive/MyDrive/bdma-07-competition/BDMA7_project_files/test_images/mistery_cat/3ad7c164-0d2f-4f38-a50c-c2b85f69fc4f.jpg: 448x640 1 Bird, 37.7ms
Speed: 3.1ms preprocess, 37.7ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)


Processing test_images/mistery_cat images: 100%|█████████▉| 399/400 [07:55<00:01,  1.13s/it]


image 1/1 /content/drive/MyDrive/bdma-07-competition/BDMA7_project_files/test_images/mistery_cat/baa5009b-b99f-418b-9231-4d387526b1f7.jpg: 416x640 1 Bird, 37.1ms
Speed: 12.7ms preprocess, 37.1ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)


Processing test_images/mistery_cat images: 100%|██████████| 400/400 [07:57<00:00,  1.19s/it]

✅ Cropped and resized images saved successfully!


In [22]:
# done